In [1]:
import pandas as pd
import os
import torch
from coords_composition_pbc import *
from sklearn.preprocessing import minmax_scale
from dataset import *
from math import sqrt

In [2]:
energy = pd.read_csv('/Users/ed_aguilar/Documents/PhD_Documents/Experiments/Metal_carbides/Experiments_periodicity/NewData/Data/energy_descriptors_all.csv', sep=',',index_col=0)
energy

,dft,c,bv,ang
original_1.vasp,-4642.635361,328,0.57782,28
original_2.vasp,-4642.616121,328,0.59434,32
original_3.vasp,-4642.593414,332,0.65032,32
original_4.vasp,-4642.634047,320,0.58117,30
original_5.vasp,-4642.573121,328,0.64733,34
...,...,...,...,...
ig_new_2.vasp,-4642.764337,300,0.31877,30
ig_new_3.vasp,-4642.771399,304,0.33505,26
ig_extra_5.vasp,-4642.779639,296,0.27402,30
ig_extra_6.vasp,-4642.782334,300,0.34220,28


In [3]:
energy['dft'] = (energy['dft']-min(energy['dft']))*13.605693122994
energy = energy['dft']

In [4]:
energy.loc['original_193.vasp']

3.592505988787065

In [5]:
energy

original_1.vasp    2.849779
original_2.vasp    3.111555
original_3.vasp    3.420498
original_4.vasp    2.867653
original_5.vasp    3.696588
                     ...   
ig_new_2.vasp      1.094973
ig_new_3.vasp      0.998883
ig_extra_5.vasp    0.886777
ig_extra_6.vasp    0.850111
ig_extra_7.vasp    1.774921
Name: dft, Length: 1046, dtype: float64

In [6]:
data = MoC_c_dir(root = '/Users/ed_aguilar/Documents/PhD_Documents/Experiments/Metal_carbides/Experiments_periodicity/NewData/Data')

In [7]:
data.len()

1046

In [15]:
df = pd.read_csv('/Users/ed/Documents/PhD_Documents/Experiments/Metal_carbides/Experiments_periodicity/NewData/Data/raw/luis_696.vasp'\
                 , header = 5)
df = df.rename(columns = {df.columns[0]: 'Mo C'})
df.head()

,Mo C
0,32 16
1,Cartesian
2,-0.02060405 0.01632000 -0.01632000
3,-0.01465168 -0.00161983 4.26789894
4,-0.01465168 4.27210106 0.00161983


In [16]:
composition = get_composition(df=df)
composition

,Mo,C
Mo C,32,16


In [17]:
Mo_y = [1 for _ in range(composition['Mo'][0])]
Mo_n = [0 for _ in range(composition['C'][0])]
Mo = Mo_y + Mo_n

In [18]:
C_n = [Mo_y[i] - 1 for i in range(len(Mo_y))]
C_y = [Mo_n[i] + 1 for i in range(len(Mo_n))]
C = C_n + C_y

In [19]:
Mo = np.expand_dims(np.asarray(Mo), axis = 1)
C = np.expand_dims(np.asarray(C), axis = 1)

In [20]:
feat = np.concatenate([Mo, C], axis = 1)
feat = torch.tensor(feat, dtype=torch.float)
feat

tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])

In [21]:
coords = get_coords(df=df)
coords[:5,:5]

array([[-2.06040500e-02,  1.63200000e-02, -1.63200000e-02],
       [-1.46516800e-02, -1.61983000e-03,  4.26789894e+00],
       [-1.46516800e-02,  4.27210106e+00,  1.61983000e-03],
       [-1.29089400e-02,  4.29371485e+00,  4.24628515e+00],
       [ 4.28551343e+00, -2.24983400e-02,  2.24983400e-02]])

In [22]:
coords = torch.tensor(coords)
coords[:3,:3]

tensor([[-2.0604e-02,  1.6320e-02, -1.6320e-02],
        [-1.4652e-02, -1.6198e-03,  4.2679e+00],
        [-1.4652e-02,  4.2721e+00,  1.6198e-03]], dtype=torch.float64)

In [23]:
torch.concat((feat, coords), axis = 1)

tensor([[ 1.0000e+00,  0.0000e+00, -2.0604e-02,  1.6320e-02, -1.6320e-02],
        [ 1.0000e+00,  0.0000e+00, -1.4652e-02, -1.6198e-03,  4.2679e+00],
        [ 1.0000e+00,  0.0000e+00, -1.4652e-02,  4.2721e+00,  1.6198e-03],
        [ 1.0000e+00,  0.0000e+00, -1.2909e-02,  4.2937e+00,  4.2463e+00],
        [ 1.0000e+00,  0.0000e+00,  4.2855e+00, -2.2498e-02,  2.2498e-02],
        [ 1.0000e+00,  0.0000e+00,  4.2823e+00,  9.1374e-03,  4.2589e+00],
        [ 1.0000e+00,  0.0000e+00,  4.2823e+00,  4.2811e+00, -9.1374e-03],
        [ 1.0000e+00,  0.0000e+00,  4.2720e+00,  4.2633e+00,  4.2767e+00],
        [ 1.0000e+00,  0.0000e+00, -7.3692e-02,  2.2084e+00,  2.1484e+00],
        [ 1.0000e+00,  0.0000e+00, -8.1123e-02,  2.1432e+00,  6.3968e+00],
        [ 1.0000e+00,  0.0000e+00, -6.2275e-02,  6.3394e+00,  2.2006e+00],
        [ 1.0000e+00,  0.0000e+00, -7.3692e-02,  6.3916e+00,  6.3316e+00],
        [ 1.0000e+00,  0.0000e+00,  4.3275e+00,  2.1874e+00,  2.2043e+00],
        [ 1.0000e+00,  0.

In [24]:
distances = get_distances_pbc(coords, 6.4)
pd.DataFrame(distances[:10,:10])

,0,1,2,3,4,5,6,7,8,9
0,NaN,2.115865,2.144302,3.012303,2.094602,2.985432,2.992804,3.676391,3.081190,2.127824
1,2.115865,NaN,3.012279,2.104777,3.008661,2.103107,3.662309,3.004132,3.062662,3.022729
2,2.144302,3.012279,NaN,2.155444,2.973626,3.685273,2.103107,2.996903,2.978430,2.129901
3,3.012303,2.104777,2.155444,NaN,3.673513,2.984186,3.004923,2.115499,2.958658,3.041991
4,2.094602,3.008661,2.973626,3.673513,NaN,2.163879,2.096595,3.012388,3.696065,2.970802
5,2.985432,2.103107,3.685273,2.984186,2.163879,NaN,3.012276,2.145948,3.669978,3.643177
6,2.992804,3.662309,2.103107,3.004923,2.096595,3.012276,NaN,2.114251,3.623438,2.952688
7,3.676391,3.004132,2.996903,2.115499,3.012388,2.145948,2.114251,NaN,3.601755,3.630257
8,3.081190,3.062662,2.978430,2.958658,3.696065,3.669978,3.623438,3.601755,NaN,2.152610
9,2.127824,3.022729,2.129901,3.041991,2.970802,3.643177,2.952688,3.630257,2.152610,NaN


In [25]:
adj = get_adj_matrix(distances, max_neighbours = 6, max_distance = 3.2)
adj.sum(axis=1)

array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6])

In [26]:
data_path = '../structures/'

In [15]:
for i in range(1, 202):
    structure = f'{i}.vasp'
    file = os.path.join(data_path, structure)
    df = pd.read_csv(file, header = 5)
    df = df.rename(columns = {df.columns[0]: 'Mo C'})
    try: 
        get_composition(df)
        coords = get_coords(df)
        distances = get_distances_pbc(coords)
        get_adj_matrix(distances, max_neighbours = 9, max_distance=6.1)
    except: 
        print('Error in file ' + file)

In [16]:
max_min_val = []
for i in range(1, 202):
    structure = f'{i}.vasp'
    file = os.path.join(data_path, structure)
    df = pd.read_csv(file, header = 5)
    df = df.rename(columns = {df.columns[0]: 'Mo C'})
 
    coords = get_coords(df)
    distances = get_distances_pbc(coords, 6.4)
    max_min_val.append(np.max(np.nanmin(distances, 0)))

In [17]:
max(max_min_val)

2.189784934948652

In [18]:
for i in range(1, 202):
    structure = f'{i}.vasp'
    file = os.path.join(data_path, structure)
    df = pd.read_csv(file, header = 5)
    df = df.rename(columns = {df.columns[0]: 'Mo C'})
    try: 
        get_composition(df)
        coords = get_coords(df)
        distances = get_distances_pbc(coords, 6.4)
        sum = get_adj_matrix(distances, max_neighbours = 26, max_distance=3.2).sum(axis=1)
        if len(sum[sum == 0 ]) != 0:
            print('Problem with file:', structure)
    except: 
        print('Error in file ' + file)

In [19]:
df = pd.read_csv('/Users/ed/Documents/PhD_Documents/Metal_carbides/structures/1.vasp', header = 5)
df = df.rename(columns = {df.columns[0]: 'Mo C'})
coords = get_coords(df=df)
distances = get_distances_pbc(coords)

In [20]:
adj = get_adj_matrix(distances, max_neighbours = 26, max_distance = 3.11)
rc = np.where(adj)
r = rc[0][0]
c = rc[1][0]
print(f'{r} is connected to {c}')

0 is connected to 8


In [21]:
c0 = coords[r]
c8 = coords[c]
print(f'{c0} \n{c8}')
d0 = - c8 + c0
print(d0)

[-0.00152964  0.00015565 -0.0222713 ] 
[-0.05539881  2.08404261  2.12956725]
[ 0.05386917 -2.08388696 -2.15183855]


In [22]:
direction_pbc(c0,c8, 6.4)

(None, 'u', 'a')

In [23]:
edge_features_pbc(c0,c8, 6.4)

array([0., 0., 1., 0., 1., 0.], dtype=float32)

In [25]:
zeros = np.zeros((48,6))
zeros[:5,:5]

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [30]:
def _get_edge_feats(df):
        edge_dist = []
        coords = get_coords(df)
        distances = get_distances_pbc(coords)
        adj = get_adj_matrix(distances, max_neighbours = 26, max_distance = 3.11)
        row,col = np.where(adj)
        edge_dir = np.zeros((len(row), 6))

        for i in range(len(row)):
            r = row[i]
            c = col[i]
            d = distances[r,c]

            c1 = coords[r]
            c2 = coords[c]
            edge_dir[i] = edge_features_pbc(c1, c2, 6.4)

            edge_dist.append(d)

        edge_dist = np.asarray(edge_dist)
        edge_dist = np.expand_dims(edge_dist, axis=1)
        edge_feats = np.concatenate([edge_dist, edge_dir], axis = 1)
        edge_feats = torch.tensor(edge_feats, dtype=torch.float)
        return edge_feats

In [32]:
print(_get_edge_feats(df=df).shape)
_get_edge_feats(df=df)

torch.Size([528, 7])


tensor([[1.5825, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0000],
        [1.6251, 1.0000, 0.0000,  ..., 0.0000, 1.0000, 0.0000],
        [1.6020, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [1.6256, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 1.0000],
        [1.6250, 0.0000, 0.0000,  ..., 1.0000, 1.0000, 0.0000],
        [1.6240, 0.0000, 1.0000,  ..., 0.0000, 1.0000, 0.0000]])

In [33]:
data[0].edge_attr

tensor([[0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 1.],
        ...,
        [1., 0., 0., 1., 0., 0.],
        [0., 1., 1., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0.]])

In [34]:
data[5].edge_index[0][:10]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [35]:
data[5].edge_attr[:5]

tensor([[0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 1.],
        [0., 1., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 1.]])

In [41]:
energy = pd.read_csv('/Users/ed/Documents/PhD_Documents/Metal_carbides/E_data/cell.features.dat', sep= ' ', usecols=[0])
energy = energy.to_numpy()
energy.shape


(201, 1)

In [45]:
data[167].y

tensor([2.5979], dtype=torch.float64)

In [44]:
e = (energy-np.min(energy))*13.605693122994
e[167]

array([2.59789907])